In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
os.environ['TF_CPP_MIN_LOG_LEVEL']='3' # only show error

In [59]:
# Define Parameters 
RAW_PATH = 'C:/Users/Zheng Shau Ming/Documents/GitHub/ML2017FALL_dataset/hw5_MF/train.csv'
OBSERVE_PATH = 'C:/Users/Zheng Shau Ming/Documents/GitHub/ML2017FALL_dataset/hw5_MF/test.csv'
SUBMIS_PATH = 'C:/Users/Zheng Shau Ming/Documents/GitHub/ML2017FALL_dataset/hw5_MF/SampleSubmisson.csv'
USER_PATH = 'C:/Users/Zheng Shau Ming/Documents/GitHub/ML2017FALL_dataset/hw5_MF/users.csv'
ITEM_PATH = 'C:/Users/Zheng Shau Ming/Documents/GitHub/ML2017FALL_dataset/hw5_MF/movies.csv'

In [60]:
# Split the raw data to training and testing set
def Train_test_split(dataset):
    TEST_SIZE = 0.3 # Parameter
    from sklearn.model_selection import train_test_split
    train, test = train_test_split(dataset, test_size=TEST_SIZE)
    return train, test

In [61]:
# Define normalize function
def Do_normalize(x, do):
    if do == 'scale':
        return (x-1)/5
    elif do == 'std':
        return (x - np.mean(x))/np.std(x)
    elif do == 'mMfs':
        return (x - np.min(x)) / (np.max(x) - np.min(x))

In [62]:
# The rmse function is for tensor type.
def rmse(y_true, y_pred):
    import keras.backend as K 
    y_pred = K.clip(y_pred, 1.0, 5.0)
    return K.sqrt(K.mean(K.pow(y_true - y_pred, 2)))

In [63]:
# Building model
def Building_new_model(MODEL,n_users, n_movies,bias=False,n_latent_factors=16,m_latent_factors=16):
    import keras
    from keras.optimizers import Adam   
    ###### Parameters #####
    WORD_LENGTH = 1 # The UserID only has one word for embedding layer.
    OPT = 'adam'
    #######################
    if MODEL == 'MF':
        movie_input = keras.layers.Input(shape=[WORD_LENGTH], name='Movie')
        movie_embedding = keras.layers.Embedding(n_movies, n_latent_factors, name='Movie-Embedding')(movie_input)
        movie_vec = keras.layers.Flatten(name='FlattenMovies')(movie_embedding)
    
        user_input = keras.layers.Input(shape=[WORD_LENGTH], name='User')
        user_embedding = keras.layers.Embedding(n_users, n_latent_factors, name='User-Embedding')(user_input)
        user_vec = keras.layers.Flatten(name='FlattenUsers')(user_embedding)
        
        prod = keras.layers.Dot(axes=1,name='DotProduct')([movie_vec, user_vec])
        add = prod
        if bias:
            movie_bias_embedding = keras.layers.Embedding(n_movies, 1, embeddings_initializer='zeros', name='Movie_bias-Embedding')(movie_input)
            movie_bias_vec = keras.layers.Flatten(name='FlattenMoviesBias')(movie_bias_embedding)

            user_bias_embedding = keras.layers.Embedding(n_users, 1, embeddings_initializer='zeros', name='User_bias-Embedding')(user_input)
            user_bias_vec = keras.layers.Flatten(name='FlattenUsersBias')(user_bias_embedding)

            add = keras.layers.Add(name='Add')([movie_bias_vec, user_bias_vec, prod])
        model = keras.Model([user_input, movie_input], add)
        
    if MODEL == 'DNN':
#        SEED = 4 # if we define a random seed for dropout, then we will get a different number 
#        after each random in dropout.
        movie_input = keras.layers.Input(shape=[WORD_LENGTH],name='Movie')
        movie_embedding = keras.layers.Embedding(n_movies,
                                                 m_latent_factors, 
                                                 name='Movie-Embedding')(movie_input)
        movie_vec = keras.layers.Flatten(name='FlattenMovies')(movie_embedding)
        movie_vec = keras.layers.Dropout(0.2)(movie_vec)

        user_input = keras.layers.Input(shape=[WORD_LENGTH], name='User')
        user_embedding = keras.layers.Embedding(n_users,
                                                 n_latent_factors, 
                                                 name='User-Embedding')(user_input)
        user_vec = keras.layers.Flatten(name='FlattenUsers')(user_embedding)
        user_vec = keras.layers.Dropout(0.02)(user_vec)

        concat = keras.layers.Concatenate(name='Concat')([movie_vec, user_vec])
        concat_dropout = keras.layers.Dropout(0.02)(concat)
        dense = keras.layers.Dense(512, name='FullyConnected', activation='relu')(concat_dropout)
        dropout_1 = keras.layers.Dropout(0.02,name='Dropout_1')(dense)
        dense_2 = keras.layers.Dense(256,name='FullyConnected-1', activation='relu')(dropout_1)
        dropout_2 = keras.layers.Dropout(0.02,name='Dropout_2')(dense_2)
        dense_3 = keras.layers.Dense(128,name='FullyConnected-2', activation='relu')(dropout_2)
        dropout_3 = keras.layers.Dropout(0.02,name='Dropout_3')(dense_3)
        dense_4 = keras.layers.Dense(64,name='FullyConnected-3', activation='relu')(dropout_3)
        dropout_4 = keras.layers.Dropout(0.02,name='Dropout_4')(dense_4)
        dense_5 = keras.layers.Dense(30,name='FullyConnected-4', activation='relu')(dropout_4)
        dropout_5 = keras.layers.Dropout(0.02,name='Dropout_4')(dense_4)
        dense_6 = keras.layers.Dense(1,name='FullyConnected-5', activation='relu')(dropout_5)   
        result = keras.layers.Dense(1, activation='relu',name='Activation')(dense_6)
        
        model = keras.Model([user_input, movie_input], result)
    model.compile(optimizer=OPT,loss= 'mean_absolute_error',metrics=['accuracy',rmse])
    model.summary()
    return model

In [64]:
def Training_model(train_dataset, model):
    from keras.callbacks import EarlyStopping, ModelCheckpoint
    ################ Parameters ################
    BATCH_SIZE = 1024
    EPOCHS = 30   
    VAL_SPLIT = 0.05
    PATIENCE = 10
    print('Number of UserID :',len(train_dataset.UserID.unique()), 
      ',Number of MovieID : ',len(train_dataset.MovieID.unique()))
    ############################################ 
    train_dataset = train_dataset.sample(frac=1) # Shuffle training data
    es = EarlyStopping(monitor='val_rmse', patience=PATIENCE, mode='min')
    history = model.fit([train_dataset.UserID, train_dataset.MovieID], train_dataset.Rating, 
                        epochs=EPOCHS, 
                        batch_size=BATCH_SIZE, 
                        validation_split=VAL_SPLIT,
                        verbose=1, callbacks=[es])
    H = history.history
    return H

In [65]:
def Plot_history(H):
    name = [['acc','val_acc','Accuracy'],['loss','val_loss','Loss'],['rmse','val_rmse','RMSE']]
    for i, j, k in name:
        see, val_see = H[i], H[j]
        plt.plot(see,'b'), plt.plot(val_see,'r')
        plt.xlabel("Epoch"), plt.ylabel(k)    
        plt.legend([i,j], loc='upper left')  
        plt.show()
        plt.close()    

In [66]:
def Evaluate_model(test_dataset, model):
    BATCH_SIZE = 1024 # Parameter
    loss, acc, rmse = model.evaluate([test_dataset.UserID, test_dataset.MovieID], 
                                       test_dataset.Rating, 
                                       batch_size=BATCH_SIZE)
    print('loss: %.5f'%loss, 'acc: %.5f'%acc, 'rmse: %.5f'%rmse)
    return np.round(loss,5), np.round(acc,5), np.round(rmse,5)

In [67]:
def Predict_and_submission(test_dataset, submission, model):
    pred_rating = model.predict([test_dataset.UserID, test_dataset.MovieID])
    pred_rating_hat = np.round(pred_rating, 0) 
    sampleSubmission = pd.read_csv(submission)
    sampleSubmission["Rating"] = pred_rating_hat
    sampleSubmission.to_csv(submission,index=None)    
    return pred_rating_hat

In [68]:
def Q1():
    MODEL = 'MF'
    N_LATENT_FACTORS = 16
    NORMALIZE = 'mMfs'
    BIAS = False
    raw_dataset = pd.read_csv(RAW_PATH) # Read raw data
    train_dataset, test_dataset = Train_test_split(raw_dataset) # Split train and test data from rawing data
    n_users,n_movies = len(train_dataset.UserID.unique()), len(train_dataset.MovieID.unique())
    
    model = Building_new_model(MODEL,n_users, n_movies,BIAS,N_LATENT_FACTORS)
    t_start = time.time()
    history = Training_model(train_dataset, model) # Training with no normalization
    print('=== Spent %s seconds ===' % np.round((time.time() - t_start), 3))
    Evaluate_model(test_dataset, model) 
    
    ###### Do normalize ######
    train_norm, test_norm = train_dataset.copy(), test_dataset.copy()
    train_norm.Rating = Do_normalize(train_dataset.Rating,NORMALIZE)
    test_norm.Rating = Do_normalize(test_dataset.Rating,NORMALIZE)
    t_start = time.time()
    model_norm = Building_new_model(MODEL,n_users,n_movies,BIAS,N_LATENT_FACTORS)
    history_norm = Training_model(train_norm, model_norm) # Training with normalization
    Evaluate_model(test_norm, model_norm)
    
    plt.plot(history["val_rmse"],label="Non-normalized RMSE")
    plt.plot(history_norm["val_rmse"],label="Normalized RMSE")
    plt.title("Nomalization comparison - Validaton")
    plt.xlabel("Epochs")
    plt.ylabel("RMSE")
    plt.legend()
    plt.show()
    plt.close()
    
    plt.plot(history["rmse"],label="Non-normalized RMSE")
    plt.plot(history_norm["rmse"],label="Normalized RMSE")
    plt.title("Nomalization comparison - Training")
    plt.xlabel("Epochs")
    plt.ylabel("RMSE")
    plt.legend()
    plt.show()
    plt.close()

In [69]:
#Q1()

In [70]:
def Q2():
    MODEL = 'MF'
    BIAS = True
    raw_dataset = pd.read_csv(RAW_PATH) # Read raw data
    train_dataset, test_dataset = Train_test_split(raw_dataset) # Split train and test data from rawing data
    n_users,n_movies = len(train_dataset.UserID.unique()), len(train_dataset.MovieID.unique())
    train_rmse, valid_rmse = [], []
    N_LATENT_FACTORS = [8,16,32,64,128,256]
    for lant in N_LATENT_FACTORS:
        model = Building_new_model(MODEL,n_users,n_movies,BIAS,lant)
        history = Training_model(train_dataset, model)
        train_rmse.append(history["rmse"])
        valid_rmse.append(history["val_rmse"])
    
    for i,j in enumerate(N_LATENT_FACTORS):
        plt.plot(valid_rmse[i],label=str(j))
    plt.title("Latent comparison - Validation")
    plt.ylim(0.75,2.2)
    plt.xlabel("Epochs")
    plt.ylabel("RMSE")
    plt.legend()
    plt.show()
    

In [71]:
#Q2()

In [72]:
def Q3():
    MODEL = 'MF'
    BIAS = False
    N_LATENT_FACTORS = 16
    raw_dataset = pd.read_csv(RAW_PATH) # Read raw data
    train_dataset, test_dataset = Train_test_split(raw_dataset) # Split train and test data from rawing data
    n_users,n_movies = len(train_dataset.UserID.unique()), len(train_dataset.MovieID.unique())
    
    model = Building_new_model(MODEL,n_users, n_movies,BIAS,N_LATENT_FACTORS)
    t_start = time.time()
    history = Training_model(train_dataset, model) # Training with no bias
    print('=== Spent %s seconds ===' % np.round((time.time() - t_start), 3))
    Evaluate_model(test_dataset, model) 
    
    BIAS = True
    model_bias = Building_new_model(MODEL,n_users, n_movies,BIAS,N_LATENT_FACTORS)
    t_start = time.time()
    history_bias = Training_model(train_dataset, model_bias) # Training with bias
    print('=== Spent %s seconds ===' % np.round((time.time() - t_start), 3))
    Evaluate_model(test_dataset, model_bias) 
    
    plt.plot(history["val_rmse"],label="No bias RMSE")
    plt.plot(history_bias["val_rmse"],label="Bias RMSE")
    plt.xlabel("Epochs")
    plt.ylabel("RMSE")
    plt.title("Bias term comparison - Validation")
    plt.legend()
    plt.show()
    plt.close() 

    plt.plot(history["rmse"],label="No bias RMSE")
    plt.plot(history_bias["rmse"],label="Bias RMSE")
    plt.title("Bias term comparison - Training")
    plt.xlabel("Epochs")
    plt.ylabel("RMSE")
    plt.legend()
    plt.show()
    plt.close()

In [73]:
#Q3()

In [74]:
def Q4():
    MODEL = 'DNN'
    BIAS = False
    N_LATENT_FACTORS_USER = 32 # DNN model
    N_LATENT_FACTORS_MOVIE = 32 # DNN model
    N_LATENT_FACTORS = 32 # MF model
    raw_dataset = pd.read_csv(RAW_PATH) # Read raw data
    train_dataset, test_dataset = Train_test_split(raw_dataset) # Split train and test data from rawing data
    n_users,n_movies = len(train_dataset.UserID.unique()), len(train_dataset.MovieID.unique())
    
    model_dnn = Building_new_model(MODEL,n_users, n_movies,BIAS,N_LATENT_FACTORS_USER,N_LATENT_FACTORS_MOVIE)
    t_start = time.time()
    history_dnn = Training_model(train_dataset, model_dnn) # Training with dnn
    print('=== Spent %s seconds ===' % np.round((time.time() - t_start), 3))
    Evaluate_model(test_dataset, model_dnn) 

    model_mf = Building_new_model(MODEL,n_users, n_movies,BIAS,N_LATENT_FACTORS)
    t_start = time.time()
    history_mf = Training_model(train_dataset, model_mf) # Training with mf
    print('=== Spent %s seconds ===' % np.round((time.time() - t_start), 3))
    Evaluate_model(test_dataset, model_mf) 
    
    plt.plot(history_dnn["val_rmse"],label="DNN RMSE")
    plt.plot(history_mf["val_rmse"],label="MF RMSE")
    plt.xlabel("Epochs")
    plt.ylabel("RMSE")
    plt.title("Bias term comparison - Validation")
    plt.legend()
    plt.show()
    plt.close() 

    plt.plot(history_dnn["rmse"],label="DNN RMSE")
    plt.plot(history_mf["rmse"],label="MF RMSE")
    plt.title("Bias term comparison - Training")
    plt.xlabel("Epochs")
    plt.ylabel("RMSE")
    plt.legend()
    plt.show()
    plt.close()
    

In [75]:
#Q4()

In [76]:
def TSNE_vis(model,emd_name,train_dataset,ItemInfo):
    movie_df = pd.read_csv(ItemInfo, sep="::")
    # Only obtain one genres
    movie_df["Genres"] = movie_df["Genres"].apply(lambda x:x.split("|")[0]) 
    movie_id = train_dataset.MovieID.unique() # To compare the movie id in the train data.
    
    # Compare and assign
    y = []
    for i in range(len(movie_id)):
        id = movie_id[i]
        genres = movie_df.loc[movie_df["movieID"]==id,"Genres"].tolist()
        y+=genres
    
    movie_emd = np.array(model.get_layer(emd_name).get_weights()).squeeze()
    from sklearn.manifold import TSNE
    tsne = TSNE(n_components=2, random_state=0, perplexity=80)
    vis_data = tsne.fit_transform(movie_emd)
    
    from sklearn import preprocessing
    le = preprocessing.LabelEncoder()
    y_c = le.fit_transform(y) # Normalize 
    plt.figure(figsize=(16,9))
    cm = plt.cm.get_cmap("tab20", 18)
    sc = plt.scatter(vis_data[:,0], vis_data[:,1], c=y_c, cmap=cm)
    plt.colorbar(ticks=range(18))
    plt.clim(-0.5, 17.5)
    plt.show()   
    pd.DataFrame({"class":list(range(18)),"genres":list(le.classes_)})


In [77]:
def Q5():
    MODEL = 'MF'
    BIAS = False
    N_LATENT_FACTORS = 16 # MF model
    raw_dataset = pd.read_csv(RAW_PATH) # Read raw data
    train_dataset, test_dataset = Train_test_split(raw_dataset) # Split train and test data from rawing data
    n_users,n_movies = len(train_dataset.UserID.unique()), len(train_dataset.MovieID.unique())
    model = Building_new_model(MODEL,n_users, n_movies,BIAS,N_LATENT_FACTORS)
    t_start = time.time()
    history = Training_model(train_dataset, model) # Training with mf
    print('=== Spent %s seconds ===' % np.round((time.time() - t_start), 3))
    Evaluate_model(test_dataset, model) 
    
    TSNE_vis(model,'Movie-Embedding',train_dataset,ITEM_PATH)

In [78]:
#Q5()

In [79]:
def Preprocess_data(dataset,UserInfo,ItemInfo,SAVE_NAME):
    user_info = pd.read_csv(UserInfo, sep="::")
    user_id = dataset.UserID
    movie_info = pd.read_csv(ItemInfo, sep="::")
    movie_info["Genres"] = movie_info["Genres"].str.split('|')
    movie_id = dataset.MovieID
    y = []        
    df = pd.DataFrame(columns=['Gender','Age','Occupation'])
    for i in range(len(dataset)):
        print("now deal with i/total ",i,"/",len(dataset))
        id = user_id[i]
        info = user_info.loc[user_info["UserID"]==id, ("Gender","Age","Occupation")]
        df = pd.concat([df,info],ignore_index=True)
        id = movie_id[i]
        genres = movie_info.loc[movie_info["movieID"]==id,"Genres"].tolist()
        y+=genres
    dataset['User_Gender'] = df.Gender.astype(bool).astype(int)
    dataset['User_Age'] = df.Age
    dataset['User_Occupation'] = df.Occupation
    dataset['Movie_Genres'] = y

    from sklearn.preprocessing import OneHotEncoder
    movie_info = pd.read_csv(ItemInfo, sep="::")
    movie_id_uni = movie_info["Genres"].apply(lambda x:x.split("|")[0])
    movie_id_uni = movie_id_uni.unique().reshape(-1,1)
    enc_m = OneHotEncoder()
    enc_m.fit(movie_id_uni)
    y = []
    for i in range(len(dataset['Movie_Genres'])):
        print("now encode i/total ",i,"/",len(dataset['Movie_Genres']))
        ec = dataset['Movie_Genres'][i]
        ec = np.asarray(ec).reshape(-1,1)
        onehot = enc_m.transform(ec).toarray().sum(axis=0)
        y.append(onehot)
    dataset['Movie_Genres'] = y     
    enc_u = OneHotEncoder()
    occ = dataset["User_Occupation"].values
    occ = enc_u.fit_transform(occ.reshape(-1,1)).toarray()
    dataset['User_Occupation'] = occ.astype(int).tolist()
    dataset.to_pickle('./'+ SAVE_NAME +'.pkl')
    return dataset

In [80]:
def Building_best_model(MODEL,n_users, n_movies,n_latent_factors=16,m_latent_factors=16):
    import keras
    from keras.optimizers import Adam
    #### Parameters ####
    WORD_LENGTH = 1 # an UserID is a value to see as a word for embedding layer.
    GENDER_LENGTH = 1
    Age_LENGTH = 1
    GENRES_LENGTH = 18
    OCCU_LENGTH = 21
    LMBDA = 0.00001
    OPT = 'adam'
    ####################
    if MODEL == 'NNMF':
        from keras.constraints import non_neg
        from keras.layers import Input, Embedding, Flatten, Dense, Dropout
        from keras.layers.merge import Dot, Add, Concatenate
        from keras.regularizers import l2
        from keras.models import Model
        # Inputs
        input_user = Input(shape=[WORD_LENGTH],name='User')
        input_movie = Input(shape=[WORD_LENGTH],name='Movie')
        input_gender = Input(shape=[GENDER_LENGTH],name='Gender')
        input_age = Input(shape=[Age_LENGTH],name='Age')
        input_occu = Input(shape=[OCCU_LENGTH],name='Occupation')
        input_genres = Input(shape=(GENRES_LENGTH,),name='Genres') 
        # Embeddings
        embedding_user = Embedding(n_users,
                                   n_latent_factors,
                                   name='NonNegUser-Embedding',
                                   embeddings_constraint=non_neg())(input_user)
        embedding_movie = Embedding(n_movies,
                                    n_latent_factors,
                                    name='NonNegMovie-Embedding',
                                    embeddings_constraint=non_neg())(input_movie)
        vec_user = Flatten(name='FlattenUsers')(embedding_user)   
        vec_movie = Flatten(name='FlattemMovies')(embedding_movie)     
        vec_occu = Dense(n_latent_factors, activation='linear')(input_occu)
        vec_gernes = Dense(n_latent_factors, activation='linear')(input_genres)
        # Dropout
        drop_user = Dropout(0.4, name='DropoutUsers')(vec_user)
        drop_movie = Dropout(0.4, name='DropoutMovies')(vec_movie)
        drop_occu = Dropout(0.4, name='DropoutOccu')(vec_occu)
        drop_gernes = Dropout(0.4, name='DropoutGernes')(vec_gernes)
        # Dot
        dot1 = Dot(axes=1,name='User-Movie')([drop_user, drop_movie])
        dot2 = Dot(axes=1,name='User-Occu')([drop_user,drop_occu])
        dot3 = Dot(axes=1,name='User-Gernes')([drop_user,drop_gernes])
        dot4 = Dot(axes=1,name='Movie-Occu')([drop_movie,drop_occu])
        dot5 = Dot(axes=1,name='Movie-Gernes')([drop_movie,drop_gernes])
        dot6 = Dot(axes=1,name='Occu-Gernes')([drop_occu,drop_gernes]) 
        # Concatenate
        con_dot = Concatenate()([dot1,dot2,dot3,dot4,dot5,dot6,input_gender,input_age])
        dense_out = Dense(1,activation='linear')(con_dot)
        # Bias
        embedding_user_bias = Embedding(n_users, 1,
                                        embeddings_initializer='zeros',
                                        name='User_bias-Embedding')(input_user)       
        embedding_movie_bias = Embedding(n_movies, 1,
                                         embeddings_initializer='zeros',
                                         name='Movie_bias-Embedding')(input_movie)
        vec_user_bias = keras.layers.Flatten(name='FlattenUsersBias')(embedding_user_bias) 
        vec_movie_bias = Flatten(name='FlattenMoviesBias')(embedding_movie_bias)
        # Output
        out = Add(name='Add')([vec_user_bias, vec_movie_bias, dense_out])            
        model = Model([input_user, input_movie, input_gender, input_age, input_occu, input_genres], out)
    
    print('model compile...')
    print('loss: mse, acc: binary, early_stopping loss: rmse') 
    model.compile(optimizer=OPT,loss= 'mean_absolute_error',metrics=['accuracy',rmse])
    model.summary()
    return model

In [81]:
def Training_best_model(train_dataset, model, now):
    from keras.callbacks import EarlyStopping, ModelCheckpoint
    ################ Parameters ################
    BATCH_SIZE = 10240
    EPOCHS = 1000
    VAL_SPLIT = 0.05
    PATIENCE = 15
    print('Number of UserID :',len(train_dataset.UserID.unique()), 
      ',Number of MovieID : ',len(train_dataset.MovieID.unique()))
    ############################################ 
    print('==================== Start Traning... ====================')
    # Shuffle training data
    train_dataset = train_dataset.sample(frac=1)
    Input_Occu = np.asarray(train_dataset["User_Occupation"].values.tolist())
    Input_Genres = np.asarray(train_dataset["Movie_Genres"].values.tolist())

    cp = ModelCheckpoint("./model/NNMF_" + str(now) + ".h5",
                     monitor='val_rmse', save_best_only=True, mode='min')
    es = EarlyStopping(monitor='val_rmse', patience=PATIENCE, mode='min')
    history = model.fit([train_dataset.UserID, train_dataset.MovieID, train_dataset.User_Gender,
                         train_dataset.User_Age,Input_Occu,Input_Genres],
                        train_dataset.Rating, 
                        epochs=EPOCHS, 
                        batch_size=BATCH_SIZE, 
                        validation_split=VAL_SPLIT,
                        verbose=1, callbacks=[cp,es])
    print('==================== END TRAINING ====================')

In [82]:
def Predict_and_submission(ob_dataset, submission, model):
    print('=== Predicting ... ===')
    Input_Occu = np.asarray(ob_dataset["User_Occupation"].values.tolist())
    Input_Genres = np.asarray(ob_dataset["Movie_Genres"].values.tolist())
    pred_rating = model.predict([ob_dataset.UserID, ob_dataset.MovieID,
                                 ob_dataset.User_Gender,ob_dataset.User_Age,
                                 Input_Occu,Input_Genres])
    pred_rating = np.clip(pred_rating,1,5) 
    print('=== Output the sample submission===')
    sampleSubmission = pd.read_csv(submission)
    sampleSubmission["Rating"] = pred_rating  
    sampleSubmission.to_csv(submission,index=None)    
    return pred_rating

In [83]:
def ensemble():
    N_LATENT_FACTORS = 128
    import pickle
    with open('preprocessing.pkl', 'rb') as file:
        train_dataset = pickle.load(file)    
    import pickle
    with open('ob_preprocessing.pkl', 'rb') as file:
        ob_dataset = pickle.load(file) 
    train_dataset['User_Age'] = Do_normalize(train_dataset['User_Age'],'std')
    ob_dataset['User_Age'] = Do_normalize(ob_dataset['User_Age'],'std')
    import keras
    for i in range(0,10):      
        model = Building_best_model('NNMF',6040,3688,N_LATENT_FACTORS)  
        Training_best_model(train_dataset, model, i)
    
    from keras.models import load_model
    model_1 = load_model('model/NNMF_0.h5', custom_objects={'rmse': rmse})
    model_2 = load_model('model/NNMF_1.h5', custom_objects={'rmse': rmse})
    model_3 = load_model('model/NNMF_2.h5', custom_objects={'rmse': rmse})
    model_4 = load_model('model/NNMF_3.h5', custom_objects={'rmse': rmse})
    model_5 = load_model('model/NNMF_4.h5', custom_objects={'rmse': rmse})
    model_6 = load_model('model/NNMF_5.h5', custom_objects={'rmse': rmse})
    model_7 = load_model('model/NNMF_6.h5', custom_objects={'rmse': rmse})
    model_8 = load_model('model/NNMF_7.h5', custom_objects={'rmse': rmse})
    model_9 = load_model('model/NNMF_8.h5', custom_objects={'rmse': rmse})
    model_10 = load_model('model/NNMF_9.h5', custom_objects={'rmse': rmse})
       
    Input_Occu = np.asarray(ob_dataset["User_Occupation"].values.tolist())
    Input_Genres = np.asarray(ob_dataset["Movie_Genres"].values.tolist())
    
    pred = []    
    pred1 = model_1.predict([ob_dataset.UserID, ob_dataset.MovieID,
                                     ob_dataset.User_Gender,ob_dataset.User_Age,
                                     Input_Occu,Input_Genres])
    pred2 = model_2.predict([ob_dataset.UserID, ob_dataset.MovieID,
                                     ob_dataset.User_Gender,ob_dataset.User_Age,
                                     Input_Occu,Input_Genres])
    pred3 = model_3.predict([ob_dataset.UserID, ob_dataset.MovieID,
                                     ob_dataset.User_Gender,ob_dataset.User_Age,
                                     Input_Occu,Input_Genres])
    pred4 = model_4.predict([ob_dataset.UserID, ob_dataset.MovieID,
                                     ob_dataset.User_Gender,ob_dataset.User_Age,
                                     Input_Occu,Input_Genres])
    pred5 = model_5.predict([ob_dataset.UserID, ob_dataset.MovieID,
                                     ob_dataset.User_Gender,ob_dataset.User_Age,
                                     Input_Occu,Input_Genres])
    pred6 = model_6.predict([ob_dataset.UserID, ob_dataset.MovieID,
                                     ob_dataset.User_Gender,ob_dataset.User_Age,
                                     Input_Occu,Input_Genres])
    pred7 = model_7.predict([ob_dataset.UserID, ob_dataset.MovieID,
                                     ob_dataset.User_Gender,ob_dataset.User_Age,
                                     Input_Occu,Input_Genres])
    pred8 = model_8.predict([ob_dataset.UserID, ob_dataset.MovieID,
                                     ob_dataset.User_Gender,ob_dataset.User_Age,
                                     Input_Occu,Input_Genres])
    pred9 = model_9.predict([ob_dataset.UserID, ob_dataset.MovieID,
                                     ob_dataset.User_Gender,ob_dataset.User_Age,
                                     Input_Occu,Input_Genres])
    pred10 = model_10.predict([ob_dataset.UserID, ob_dataset.MovieID,
                                     ob_dataset.User_Gender,ob_dataset.User_Age,
                                     Input_Occu,Input_Genres])
    
    pred = np.average(np.concatenate((pred1,pred2,pred3,pred4,pred5,pred6,pred7,
                                      pred8,pred9,pred10),axis=-1),axis=-1)
    pred = np.clip(pred,1,5) 
    sampleSubmission = pd.read_csv(SUBMIS_PATH)
    sampleSubmission["Rating"] = pred  
    sampleSubmission.to_csv(SUBMIS_PATH,index=None)    

In [84]:
ensemble()

model compile...
loss: mse, acc: binary, early_stopping loss: rmse
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
User (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
Movie (InputLayer)              (None, 1)            0                                            
__________________________________________________________________________________________________
NonNegUser-Embedding (Embedding (None, 1, 128)       773120      User[0][0]                       
__________________________________________________________________________________________________
NonNegMovie-Embedding (Embeddin (None, 1, 128)       472064      Movie[0][0]                      
__________________________________________

854879/854879 [==============================] - 20s 23us/step - loss: 3.1922 - acc: 0.0466 - rmse: 2.5833 - val_loss: 1.3040 - val_acc: 0.2230 - val_rmse: 1.5534
Epoch 2/1000
854879/854879 [==============================] - 10s 12us/step - loss: 1.0340 - acc: 0.3014 - rmse: 1.2590 - val_loss: 0.7857 - val_acc: 0.3923 - val_rmse: 0.9974
Epoch 3/1000
854879/854879 [==============================] - 10s 12us/step - loss: 0.8667 - acc: 0.3583 - rmse: 1.0813 - val_loss: 0.7543 - val_acc: 0.4118 - val_rmse: 0.9619
Epoch 4/1000
854879/854879 [==============================] - 10s 12us/step - loss: 0.8485 - acc: 0.3668 - rmse: 1.0604 - val_loss: 0.7462 - val_acc: 0.4132 - val_rmse: 0.9525
Epoch 5/1000
854879/854879 [==============================] - 10s 12us/step - loss: 0.8371 - acc: 0.3718 - rmse: 1.0477 - val_loss: 0.7434 - val_acc: 0.4158 - val_rmse: 0.9475
Epoch 6/1000
854879/854879 [==============================] - 10s 12us/step - loss: 0.8299 - acc: 0.3753 - rmse: 1.0396 - val_loss: 0

Train on 854879 samples, validate on 44994 samples
Epoch 1/1000
854879/854879 [==============================] - 11s 12us/step - loss: 1.7319 - acc: 0.1782 - rmse: 1.8861 - val_loss: 0.8756 - val_acc: 0.3442 - val_rmse: 1.0842
Epoch 2/1000
854879/854879 [==============================] - 10s 11us/step - loss: 0.8828 - acc: 0.3468 - rmse: 1.0962 - val_loss: 0.7612 - val_acc: 0.4017 - val_rmse: 0.9623
Epoch 3/1000
854879/854879 [==============================] - 10s 12us/step - loss: 0.8303 - acc: 0.3712 - rmse: 1.0393 - val_loss: 0.7402 - val_acc: 0.4152 - val_rmse: 0.9408
Epoch 4/1000
854879/854879 [==============================] - 10s 12us/step - loss: 0.8080 - acc: 0.3815 - rmse: 1.0141 - val_loss: 0.7312 - val_acc: 0.4215 - val_rmse: 0.9326
Epoch 5/1000
854879/854879 [==============================] - 10s 11us/step - loss: 0.7936 - acc: 0.3899 - rmse: 0.9984 - val_loss: 0.7275 - val_acc: 0.4250 - val_rmse: 0.9286
Epoch 6/1000
854879/854879 [==============================] - 10s 11u

Epoch 47/1000
854879/854879 [==============================] - 10s 11us/step - loss: 0.6672 - acc: 0.4751 - rmse: 0.8647 - val_loss: 0.6856 - val_acc: 0.4555 - val_rmse: 0.8850
Epoch 48/1000
854879/854879 [==============================] - 10s 12us/step - loss: 0.6649 - acc: 0.4780 - rmse: 0.8628 - val_loss: 0.6856 - val_acc: 0.4567 - val_rmse: 0.8852
Epoch 49/1000
854879/854879 [==============================] - 10s 12us/step - loss: 0.6630 - acc: 0.4793 - rmse: 0.8607 - val_loss: 0.6847 - val_acc: 0.4560 - val_rmse: 0.8844
Epoch 50/1000
854879/854879 [==============================] - 10s 12us/step - loss: 0.6612 - acc: 0.4807 - rmse: 0.8589 - val_loss: 0.6838 - val_acc: 0.4586 - val_rmse: 0.8841
Epoch 51/1000
854879/854879 [==============================] - 12s 14us/step - loss: 0.6585 - acc: 0.4831 - rmse: 0.8563 - val_loss: 0.6833 - val_acc: 0.4597 - val_rmse: 0.8841
Epoch 52/1000
854879/854879 [==============================] - 10s 12us/step - loss: 0.6576 - acc: 0.4839 - rmse: 0

Train on 854879 samples, validate on 44994 samples
Epoch 1/1000
854879/854879 [==============================] - 12s 15us/step - loss: 2.0797 - acc: 0.1389 - rmse: 2.1184 - val_loss: 1.0314 - val_acc: 0.3060 - val_rmse: 1.2871
Epoch 2/1000
854879/854879 [==============================] - 11s 13us/step - loss: 0.9267 - acc: 0.3385 - rmse: 1.1562 - val_loss: 0.7845 - val_acc: 0.4010 - val_rmse: 1.0016
Epoch 3/1000
854879/854879 [==============================] - 11s 13us/step - loss: 0.8378 - acc: 0.3726 - rmse: 1.0519 - val_loss: 0.7585 - val_acc: 0.4127 - val_rmse: 0.9691
Epoch 4/1000
854879/854879 [==============================] - 11s 12us/step - loss: 0.8158 - acc: 0.3822 - rmse: 1.0256 - val_loss: 0.7485 - val_acc: 0.4164 - val_rmse: 0.9559
Epoch 5/1000
854879/854879 [==============================] - 12s 14us/step - loss: 0.8031 - acc: 0.3877 - rmse: 1.0101 - val_loss: 0.7418 - val_acc: 0.4197 - val_rmse: 0.9477
Epoch 6/1000
854879/854879 [==============================] - 10s 12u

...

854879/854879 [==============================] - 10s 11us/step - loss: 0.6391 - acc: 0.5008 - rmse: 0.8358 - val_loss: 0.6856 - val_acc: 0.4566 - val_rmse: 0.8834
Epoch 94/1000
854879/854879 [==============================] - 9s 11us/step - loss: 0.6386 - acc: 0.5015 - rmse: 0.8357 - val_loss: 0.6857 - val_acc: 0.4557 - val_rmse: 0.8829
==================== END TRAINING ====================
model compile...
loss: mse, acc: binary, early_stopping loss: rmse
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
User (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
Movie (InputLayer)              (None, 1)            0                                            
____________________________________________

Train on 854879 samples, validate on 44994 samples
Epoch 1/1000
854879/854879 [==============================] - 11s 13us/step - loss: 3.1169 - acc: 0.0574 - rmse: 2.5475 - val_loss: 1.2991 - val_acc: 0.2354 - val_rmse: 1.5550
Epoch 2/1000
854879/854879 [==============================] - 10s 11us/step - loss: 1.0556 - acc: 0.2975 - rmse: 1.2805 - val_loss: 0.7965 - val_acc: 0.3883 - val_rmse: 1.0078
Epoch 3/1000
854879/854879 [==============================] - 10s 11us/step - loss: 0.8612 - acc: 0.3602 - rmse: 1.0749 - val_loss: 0.7489 - val_acc: 0.4135 - val_rmse: 0.9539
Epoch 4/1000
854879/854879 [==============================] - 10s 11us/step - loss: 0.8351 - acc: 0.3725 - rmse: 1.0451 - val_loss: 0.7388 - val_acc: 0.4212 - val_rmse: 0.9429
Epoch 5/1000
854879/854879 [==============================] - 10s 11us/step - loss: 0.8230 - acc: 0.3783 - rmse: 1.0308 - val_loss: 0.7349 - val_acc: 0.4210 - val_rmse: 0.9373
Epoch 6/1000
854879/854879 [==============================] - 10s 11u

854879/854879 [==============================] - 10s 12us/step - loss: 0.7124 - acc: 0.4419 - rmse: 0.9126 - val_loss: 0.7063 - val_acc: 0.4413 - val_rmse: 0.9033
Epoch 48/1000
854879/854879 [==============================] - 10s 12us/step - loss: 0.7107 - acc: 0.4428 - rmse: 0.9112 - val_loss: 0.7054 - val_acc: 0.4418 - val_rmse: 0.9035
Epoch 49/1000
854879/854879 [==============================] - 10s 12us/step - loss: 0.7079 - acc: 0.4448 - rmse: 0.9080 - val_loss: 0.7057 - val_acc: 0.4419 - val_rmse: 0.9037
Epoch 50/1000
854879/854879 [==============================] - 10s 12us/step - loss: 0.7062 - acc: 0.4459 - rmse: 0.9067 - val_loss: 0.7057 - val_acc: 0.4414 - val_rmse: 0.9038
Epoch 51/1000
854879/854879 [==============================] - 10s 12us/step - loss: 0.7043 - acc: 0.4475 - rmse: 0.9048 - val_loss: 0.7042 - val_acc: 0.4437 - val_rmse: 0.9031
Epoch 52/1000
854879/854879 [==============================] - 10s 12us/step - loss: 0.7022 - acc: 0.4492 - rmse: 0.9027 - val_lo

Train on 854879 samples, validate on 44994 samples
Epoch 1/1000
854879/854879 [==============================] - 12s 14us/step - loss: 2.5113 - acc: 0.1085 - rmse: 2.2519 - val_loss: 1.1342 - val_acc: 0.2736 - val_rmse: 1.3884
Epoch 2/1000
854879/854879 [==============================] - 10s 12us/step - loss: 1.0602 - acc: 0.2904 - rmse: 1.2880 - val_loss: 0.8141 - val_acc: 0.3747 - val_rmse: 1.0189
Epoch 3/1000
854879/854879 [==============================] - 10s 12us/step - loss: 0.9083 - acc: 0.3397 - rmse: 1.1229 - val_loss: 0.7489 - val_acc: 0.4058 - val_rmse: 0.9447
Epoch 4/1000
854879/854879 [==============================] - 10s 12us/step - loss: 0.8673 - acc: 0.3565 - rmse: 1.0768 - val_loss: 0.7353 - val_acc: 0.4150 - val_rmse: 0.9306
Epoch 5/1000
854879/854879 [==============================] - 10s 12us/step - loss: 0.8478 - acc: 0.3649 - rmse: 1.0554 - val_loss: 0.7297 - val_acc: 0.4211 - val_rmse: 0.9264
Epoch 6/1000
854879/854879 [==============================] - 10s 12u

854879/854879 [==============================] - 10s 12us/step - loss: 0.7250 - acc: 0.4320 - rmse: 0.9285 - val_loss: 0.7174 - val_acc: 0.4323 - val_rmse: 0.9190
Epoch 48/1000
854879/854879 [==============================] - 10s 12us/step - loss: 0.7238 - acc: 0.4325 - rmse: 0.9272 - val_loss: 0.7171 - val_acc: 0.4340 - val_rmse: 0.9195
Epoch 49/1000
854879/854879 [==============================] - 10s 12us/step - loss: 0.7228 - acc: 0.4336 - rmse: 0.9263 - val_loss: 0.7173 - val_acc: 0.4319 - val_rmse: 0.9186
Epoch 50/1000
854879/854879 [==============================] - 10s 12us/step - loss: 0.7220 - acc: 0.4337 - rmse: 0.9255 - val_loss: 0.7175 - val_acc: 0.4324 - val_rmse: 0.9196
Epoch 51/1000
854879/854879 [==============================] - 10s 12us/step - loss: 0.7211 - acc: 0.4340 - rmse: 0.9246 - val_loss: 0.7175 - val_acc: 0.4327 - val_rmse: 0.9193
Epoch 52/1000
854879/854879 [==============================] - 10s 12us/step - loss: 0.7203 - acc: 0.4347 - rmse: 0.9239 - val_lo